# Hopfield Network With Hashing - Dayan & Abbott

The following code takes as input .wav sound files and transforms each of them into MFCC vectors. These vectors are then each transformed into a hash. They are then used to train a Hopfield network. In this way, each sound vector becomes a memory pattern that can be accessed even if slightly corrupted.

This is a memory mechanism in a form of a Hopfield network. The stored items are called memory patterns. They are retrieved by a process of the input that is presented to the network dynamics which at some time step reaches a fixed stable point. This means that the input item has been recognized (i.e. there is a memory pattern identical or very similar to it).

Even noisy sounds or those corrupted to some extent can be accessed. In other words, if the input is $x_1 + \delta$ and the stored item is $x_1$, the network will still reach the fixed point of $x_1$ if $\delta$ is small enough.

Additionally, for storage purposes, sounds are transformed each into a hash - with this we reduce their dimensionality. This means we increase the storage capacity. 

### Load dependencies

In [2]:
# First, we load some dependencies.
import numpy as np
import math
import librosa
import scipy.io.wavfile as wav
import sys
import glob
import random

In [3]:
# Folder with some wav files to test this script.
folder_train = "./wavs/"

### Extract Features

First, we need to transform the sounds into a readable format (here, MFCCs) that can be used for hashing.

In [25]:
def make_mfcc(folder):
    """
    Go through the folder and find all (and only) files ending with .wav
    Here, we transform each .wav file into MFCCs and then flatten them into one vector.
    We do this because we want one hash per .wav file.
    
    Any file shorter than the longest file in the folder will be padded with values 0,
    so that all concatenated file vectors are of the same length.
    
    Parameters
    ----------
    folder : path to folder with wav sounds
    
    Returns
    -------
    a list of flattened MFCC vectors
    """
    vectors = []
    for file in glob.glob(folder + "*.wav", recursive=True):
        y, sr = librosa.load(file)
        mfcc_feat = librosa.feature.mfcc(y=y, sr=sr)
        vect = mfcc_feat.flatten()
        vectors.append(vect)
    # find the largest vector
    max_length = len(max(vectors, key=lambda p: len(p)))
    # append zeros to all the other vectors
    for i in range(len(vectors)):
        vectors[i] = np.pad(vect, (0,max_length-len(vect)))
    return vectors

### Hashing of features

Now we will use these features and transform them into hash vectors, which we will use to store in our memory. We do this to facilitate memory storage: hashes are vectors with reduced dimensionality, with values mostly equal to 0 and a few of them equal to 1.

In [12]:
def hash_dim(d,k,m,seed):  
    
    """
    Define hash parameters.
    The hash will be a matrix of the dimension = k*m
    We choose a random number k of units of the vector.
    
    Parameters
    ----------
    d : num
        Length of a random vector being stored
    k : num
        Number of units we randomly choose of the vector
    m : num
        Number of times we will  do the hashing for some vector
    seed : num
        We always want the same units randomly chosen
        
    Returns
    -------
    a numpy array 
        p of dimensions [k,m] represents randomly chosen dimensions
    
    """   
    
    assert k <= d
    p = np.zeros((m,k,))
    np.random.seed(seed)
    for i in range(m):
        p[i] = np.random.permutation(d)[:k]
    return p

    
def get_hash(vector, k, m, p): 
    """
    Transform a vector of speech into a hash
    The hash will be a matrix of the dimension = k*m
    
    Once we have chosen k random dimensions, we look for the highest 
    value and turn it into 1. Everything else is 0.
    We thus get sparse matrices.
    We do this m times. Final output is h=k*m.
    
    Parameters
    ----------
    vector : np.array
        Features (i.e. MFCC) of some sound with dim = 1*n
    k : num
        Number of units we randomly choose of the vector
    m : num
        Number of times we will do the hashing for some vector.
    p : numpy array
        p of dimensions [k,m] represents randomly chosen dimensions
        
    Returns
    -------
    a numpy array h of size [1, k*m]
    """
    
    h = np.zeros((m,k,))
    for i in range(m):
        p_line = p[i]
        ix = np.argmax(vector[p_line])
        hi = np.zeros(k)
        hi[ix] = 1
        h[i] = hi
    h = np.hstack(h)
    return h

In [13]:
# TEST

expected_h = np.array([[1,0,0],[0,0,1]]).flatten()
vector = np.array([6,4,5,9,2])
p0 = hash_dim(len(vector),3,2,2).astype(int)
print("This is a test hash: ", get_hash(vector, 3, 2, p0))
assert get_hash(vector, 3, 2, p0).all() == expected_h.all()

This is a test hash:  [1. 0. 0. 0. 0. 1.]


### Memory storage

Hopfield network consists of a dynammic network where we can store memories. In particular, the storage is the symmetric recurrent weight matrix that is trained with memory patterns (presented as hash vectors) we are storing. This results with each of them becoming a fixed point of the network. Once we want to "retrieve" a memory pattern, we need to find one of the fixed points. 

In [14]:
def get_m(lmbda, alpha, c, N, V):
    """
    Obtain the matrix M (symmetric recurrent weight matrix) representing memory storage.
    
    Parameters
    ----------
    lmbda : num
        Eigenvalue represented as a lambda
    alpha : num
        Number representing the amount of active units
    c : num
        Constant value of active components, inactive have 0
    N : num
        Number of neurons used
    V : list
        A list of vectors in a hashed form
        
    Returns
    -------
    a numpy array m 
    """
    # n is a vector of ones
    n = np.ones(N)

    vect_sum = np.zeros((N,N))
    for vect in V:
        outer_prod = np.outer((vect - alpha * c * n),(vect - alpha * c * n))
        vect_sum += outer_prod

    m = (lmbda / (pow(c,2)*alpha*N*(1-alpha))) \
        * vect_sum \
        - (np.outer(n,n) / (alpha*N))
    return m

We need to determine what is a fixed point of the network. That is, we want to know when the system is stable. When the system is stable, this indicated we have retrieved a memory stored in our network.  

In [15]:
def has_converged(x0, x1, tau):
    """
    Decides whether the system has converged.
    
    Parameters
    ----------
    x0 : num
        A point from the moment t-1
    x1 : num
        Updated point from the moment t
    tau : num
        Number representing the threshold
        
    Returns
    -------
    boolean : True or False
    """
#     return math.isclose(x0, x1, rel_tol=tau) 
    return np.allclose(x0, x1, rtol=tau)

def fixed_point(x0, m, tau, i):
    """
    Decides whether a fixed point of the system was reached.
    This means we have retrieved a memory.
    Memory pattern satisfies v_m = F(M * v_m) (i.e. is a fixed point)
    We use a sigmoid function as F: F = 1/(1+np.exp(-x))
    
    Parameters
    ----------
    x0 : num
        A point we start evaluating the system from
    m : numpy array
        Symmetric recurrent weight matrix (memory storage)
    tau : num
        Number representing the threshold
    i : num
        Index at which we start
        
    Returns
    -------
    x1 : fixed point of the system, given that we started at x0
    """
    x1 = 1/(1 + np.exp(- np.inner(m,x0) ))
    if not has_converged(x0, x1, tau):
        print("Step: ", i)
        i += 1
        return fixed_point(x1, m, tau, i)
    else:
        return x1

#     if F == None:
#         x1 = 1/(1 + np.exp(- np.inner(m,x0) ))
#     else:
#         x1 = F*x0
    
#     if convergence_criterion(x0, x1, tau):
#         return x1
#     else:
#         i += 1
#         print(i)
#         return fixed_point(x1, tau, F)

In [16]:
# Test:

k = 5
m = 3
lmbda = 0.1
alpha = 0.6
c = 1
N = 15
V =[]

vect_length = 15

p = hash_dim(vect_length,k,m,27).astype(int)
mfccs_vectors = make_mfcc(folder_train)
for vect in mfccs_vectors:
    v = get_hash(vect, k, m, p)
    V.append(v)

M = get_m(lmbda, alpha, c, N, V)


In [17]:
# Test to get to fixed poind

tau1 = 0.000001
tau2 = 0.1
# x0_1 is a vector similar to a memory pattern that is stored
x0_1 = V[0]
print(V[0])
# x0_2 is a vector very different from any memory pattern stored
x0_2 = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

print(fixed_point(x0_1, M, tau1, 0))

print(fixed_point(x0_1, M, tau2, 0))

# This is problematic: x0_2 is not even nearly similar to any of the stored patterns, 
# yet w reach the local minimum/fixed point

print(fixed_point(x0_2, M, tau1, 0))

print(fixed_point(x0_2, M, tau2, 0))

[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
Step:  0
Step:  1
Step:  2
[1.         1.         1.         0.99998091 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.        ]
Step:  0
Step:  1
[1.         1.         1.         0.99998092 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.        ]
Step:  0
[1.         1.         1.         0.99998091 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.        ]
[1.         1.         1.         0.99998092 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.        ]


In [18]:
print(M)

[[10.74777778  6.29222222  6.74222222 -2.34111111  6.45888889  8.41444444
   5.51444444  6.99777778  0.08111111  6.89222222  8.28111111  6.88666667
   0.13111111  7.50333333  5.09777778]
 [ 6.29222222 11.19777778  8.09222222 -0.99111111  7.80888889  6.76444444
   8.94777778  7.95888889  1.29222222  7.43666667  7.82555556  8.90333333
   0.23111111  8.82555556  6.61444444]
 [ 6.74222222  8.09222222 11.34777778 -0.54111111  8.25888889  7.29777778
   7.89777778  8.71444444  1.93666667  8.05333333  7.19222222  9.82555556
   0.45888889  9.49777778  6.92555556]
 [-2.34111111 -0.99111111 -0.54111111  8.32       -0.82444444 -0.81333333
  -1.18555556  0.90888889  5.54777778 -0.83555556 -1.25222222  0.04777778
   5.62555556  0.49777778 -1.29666667]
 [ 6.45888889  7.80888889  8.25888889 -0.82444444 11.25333333  7.34777778
   7.61444444  8.32        1.82        7.85333333  7.07555556  8.62555556
   0.23111111  9.74222222  7.28111111]
 [ 8.41444444  6.76444444  7.29777778 -0.81333333  7.34777778 10.

In [20]:
import pickle
with open("test.txt", "wb") as fp:
    pickle.dump(mfccs_vectors, fp)

In [38]:
len(max(mfccs_vectors, key=lambda p: len(p)))

8232140